## Imports

In [ ]:
import pandas as pd
import numpy as np
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import warnings
from sklearn.model_selection import train_test_split
from pathlib import Path
# another way of ignoring warnings
warnings.filterwarnings('ignore')

## Load dataset

In [ ]:
path = Path().cwd()
data_file = path.parent / "Data" / "ClevelandHeartData.csv"

T = pd.read_csv(data_file, sep=';')
X = T.loc[:, T.columns != 'Diagnosis']
y = np.array(T.loc[:, T.columns == 'Diagnosis'])

# Some of the data is written in a European format, that doesn't work in python
X['Oldpeak'] = pd.Series(X['Oldpeak']).str.replace(',', '.')
# Decision trees in python don't handle NaN values in the same way as in matlab
# This model does not support missing values
# we replace every empty value with the column's mode since we have categorical 
for column in X.columns:
    X[column].fillna(X[column].mode()[0], inplace=True)


Xlabel = np.array(T.columns[:-1])
Nobs = len(T)
Nfeature = len(Xlabel)
print(f'Number of observations: {Nobs}.')
print(f'Number of features: {Nfeature}.')
print(f'Labels of features: {Xlabel}')
print(T)

### 2. Matlab and R have functions for building, pruning, evaluating and viewing classification and regression trees. We are going touse them to diagnose heart problems based on a set of 13 clinical variables from 303 patients and healthy controls. The data is in the file ClevelandHeartData.csv. The first 13 columns are dierent features and the 14th column is an indicator for heart problem/healthy. You can read more about the data in ClevelandHeartDataDescription.txt.

> a) Read the help files for the tree methods in your preferred language to familiarize yourself with the possibilities.

Read about options in the DecisionTreeClassifier at http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier


> b) Build a large tree with the minimum number of observations (minLeaf ) in at leaf
set to 1 and view the tree.

In [ ]:
# Build and view a large tree
dtree=



fig, axes = plt.subplots(nrows = 1,ncols = 1,figsize = (10,10), dpi=300)
tree.plot_tree(dtree,
               feature_names = Xlabel,
               class_names = ['No Diagnose','Diagnose'],
               filled = True);

#A little description of the information at each plotted node
#1. row: The condition
#2. row: The impurity score of the node
#3. row: The number of observations at this node
#4. row: The number of samples for each class at this node
#5. row: The class by majority voting

> c) Choose optimal tree size by tuning the parameter MinLeaf value using cross
validation.

In [ ]:
# use cross validation to tune the hyper parameter for nim_sample_leaf
# use sklearns GridSearchCV
dtree = 

cv_grid = 

# Fit the grid search model


#plot the results
plt.plot(range(1,50),cv_grid.cv_results_['mean_test_score'])
plt.xlabel('Min_samples_leaf')
plt.ylabel('Testaccuracy')
print(f'best estimator: {cv_grid.best_estimator_}')

In [ ]:
# One standard error rule
meanError = list(cv_grid.cv_results_['mean_test_score'])
stdError = cv_grid.cv_results_['std_test_score']
# this is the index of the smallest error

# the model picks model with highest accuracy and because of that we use max
maxAcc = meanError.index(max(meanError))
 # model does accuracy and not error so we flip shit!
J = np.where(meanError[maxAcc] - stdError[maxAcc] < meanError)[0]

if (len(J) > 0):
    j = int(J[-1::])
else:
    j = minError

min_sample_leaf_opt = j+1
print (f'One standard error rule gives min_samples_leaf: {min_sample_leaf_opt}') # +1 because zero indexed

> d) View the optimal tree and try to interpret it such that it makes sense for a doctor.

In [ ]:
# Build a tree based on best nim_sample_leaf
dtree=



fig, axes = plt.subplots(nrows = 1,ncols = 1,figsize = (10,10), dpi=300)
tree.plot_tree(dtree,
               feature_names = Xlabel,
               class_names = ['No Diagnose','Diagnose'],
               filled = True);

## If you have scikit-learn 0.22 <= then you can also find the tree size through cost complexity pruning of the best estimator

In [ ]:
# here we prune the tree instead of using the one std error because the tree becomes too small
# use cross validation to minimize cost complexity pruning

dtree =

cv_grid = 

# Fit the grid search model


print(f'best estimator: {cv_grid.best_estimator_}')

In [ ]:
# Build a tree based on best pruning params
dtree=


fig, axes = plt.subplots(nrows = 1,ncols = 1,figsize = (10,10), dpi=300)
tree.plot_tree(dtree,
               feature_names = Xlabel,
               class_names = ['No Diagnose','Diagnose'],
               filled = True);